# Case
## Imports

In [1]:
import cadquery as cq
from cadquery import exporters, Sketch, Location, Vector

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay

enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

print()
versions()

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay

jupyter_cadquery  3.1.0rc4
cad_viewer_widget 1.3.2
OCP               7.5.3


## Viewer

In [2]:
cv = open_viewer("Assembly", cad_width=850, height=1240)
set_defaults(viewer="Assembly")

## Constants
### General

In [3]:
BOTTOM_PLATE_HEIGHT = 2
BOTTOM_PLATE_INDENTATION = 1
PCB_BOTTOM_SPACING = 2.4
PCB_HEIGHT = 1.6
PCB_SWITCH_PLATE_SPACING = 0.9
SWITCH_PLATE_HEIGHT = 1.2
SWITCH_BORDER_HEIGHT = 5

HOLES = [((96.9, 107.88), 2, 0), 
         ((175.28, 86.28), 2, 0), 
         ((135.05, 37), 3, BOTTOM_PLATE_HEIGHT + 3)]

### Wired Case

In [4]:
OLED_SPACING = 8
ACRYLIC_GLASS_HEIGHT = 3
WIRED_MCU_USB_OFFSET = 2.5

### Wireless Case

In [5]:
LCD_SPACING = 5.5
LCD_COVER_HEIGHT = 0.8
WIRELESS_MCU_USB_OFFSET = 4

## Helper Functions

### Generell

In [6]:
def createCaseOutline():
    return cq.Workplane("XY")\
    .moveTo(78.77, -42.15)\
    .lineTo(78.77, -90.9)\
    .radiusArc((75.91, -96.0), 8)\
    .lineTo(70.57, -99.08)\
    .radiusArc((68.98, -105.04), -4.3)\
    .lineTo(82.05, -127.54)\
    .radiusArc((88.00, -129.13), -4.3)\
    .radiusArc((118.04, -116.55), 79.9)\
    .lineTo(176.41, -116.55)\
    .radiusArc((179.39, -113.61), -2.9)\
    .lineTo(179.39, -110.34)\
    .radiusArc((182.34, -107.39), 2.9)\
    .lineTo(194.62, -107.39)\
    .radiusArc((197.6, -104.45), -2.9)\
    .lineTo(197.6, -51.11)\
    .radiusArc((194.66, -48.13), -2.9)\
    .lineTo(182.34, -48.14)\
    .radiusArc((179.4, -45.16), 2.9)\
    .lineTo(179.4, -41.12)\
    .radiusArc((177.15, -37.56), -3.7)\
    .radiusArc((103.15, -37.15), -104.25)\
    .lineTo(83.77, -37.15)\
    .radiusArc((78.77, -42.15), -5)\
    .close()

def createBottomPlate():
    return createCaseOutline()\
        .faces()\
        .offset2D(-3)\
        .extrude(BOTTOM_PLATE_HEIGHT)

def extrudeDisplayCover(case, offset):
    workplane = case.faces(">Z").workplane().moveTo(78.77, -42.15)\
        .lineTo(78.77, -90.9)\
        .lineTo(103.15, -90.9)\
        .lineTo(103.15, -37.15)\
        .lineTo(83.77, -37.15)\
        .radiusArc((78.77, -42.15), -5)\
        .close()\
        .workplane(offset=offset)
    return workplane.moveTo(82.77, -42.15)\
    .lineTo(82.77, -85.9)\
    .lineTo(103.15, -85.9)\
    .lineTo(103.15, -39.15)\
    .lineTo(83.77, -39.15)\
    .radiusArc((82.77, -42.15), -5)\
    .close()\
    .loft(combine=True)

def cutBottomPlate(case):
    return case.cut(createBottomPlate())

def cutSwitchHoles(case):
    innerZOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT + PCB_SWITCH_PLATE_SPACING
    innerCutout = cq.importers.importDXF('switch_outline.dxf', tol=1e-3)\
        .wires()\
        .toPending()\
        .faces()\
        .extrude(SWITCH_PLATE_HEIGHT)\
        .translate((0, 0, innerZOffset))
    outerZOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT
    outerCutout = cq.importers.importDXF('switch_outline.dxf', tol=1e-3)\
        .wires()\
        .toPending()\
        .faces()\
        .offset2D(1)\
        .extrude(PCB_SWITCH_PLATE_SPACING)\
        .translate((0, 0, outerZOffset))
    return case.cut(innerCutout).cut(outerCutout)

def cutKeycapsBorder(case):
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT + SWITCH_PLATE_HEIGHT + PCB_SWITCH_PLATE_SPACING
    cutout = cq.importers.importDXF('keycaps_outline.dxf', tol=1e-3)\
        .wires()\
        .toPending()\
        .faces()\
        .extrude(SWITCH_BORDER_HEIGHT + 5)\
        .translate((0, 0, zOffset))
    return case.cut(cutout)

def cutTrackPointShieldIndentation(case):
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT 
    workplane = case.faces("<Z").workplane()
    mainCutout = workplane.moveTo(137.93, 93.41)\
        .lineTo(137.93, 111.29)\
        .lineTo(174.16, 111.29)\
        .lineTo(174.16, 93.41)\
        .close()\
        .extrude(-3, combine=False)\
        .translate((0, 0, zOffset))
    con1Cutout = workplane.moveTo(133, 90)\
        .lineTo(133, 111.29)\
        .lineTo(138, 111.29)\
        .lineTo(138, 90)\
        .close()\
        .extrude(-PCB_SWITCH_PLATE_SPACING, combine=False)\
        .translate((0, 0, zOffset))
    con2Cutout = workplane.moveTo(133, 90)\
        .lineTo(133, 111.29)\
        .lineTo(164, 111.29)\
        .lineTo(164, 90)\
        .close()\
        .extrude(-PCB_SWITCH_PLATE_SPACING, combine=False)\
        .translate((0, 0, zOffset))
    return case.cut(mainCutout).cut(con1Cutout).cut(con2Cutout)

def cutPcb(case):
    zOffset = BOTTOM_PLATE_HEIGHT
    cutout = cq.importers.importDXF('pcb_outline.dxf', tol=1e-3)\
        .wires()\
        .toPending()\
        .faces()\
        .extrude(PCB_BOTTOM_SPACING + PCB_HEIGHT)\
        .translate((0, 0, zOffset))
    return case.cut(cutout)


def cutUsbCConnector(case, usbPortOffset, outerDepth = -2.5):
    zOrigin = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + usbPortOffset
    workplane = case.faces("<Y[3]").workplane(centerOption="ProjectedOrigin", origin=(92.41, -41.75, zOrigin))
    inner = workplane.rect(9, 3.5).extrude(-6, combine=False).edges("-Y").fillet(1.7)
    outer = workplane.rect(12, 7).extrude(outerDepth, combine=False).edges("-Y").fillet(3.4)
    return case.cut(inner).cut(outer)

def cutMountingHoles(case):
    for hole in HOLES:
        case = cutHole(case, hole[0], diameter = hole[1], depth = hole[2])
    return case

def cutTrackpointHole(case):
    result = cutHole(case,   (121.23, 58.4), diameter = 2)
    result = cutHole(result, (121.23, 49.9), diameter = 2)
    result = cutHole(result, (121.23, 67.93), diameter = 2)
    return result

def cutHole(case, pnt, diameter = 2, depth = 0):
    defaultDepth = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT + SWITCH_PLATE_HEIGHT + PCB_SWITCH_PLATE_SPACING
    d = defaultDepth if depth == 0 else depth
    return case.faces("<Z").workplane().moveTo(pnt[0], pnt[1]).hole(diameter, d)

def createBaseCase():
    height = SWITCH_BORDER_HEIGHT\
            + SWITCH_PLATE_HEIGHT\
            + PCB_SWITCH_PLATE_SPACING\
            + PCB_HEIGHT\
            + PCB_BOTTOM_SPACING\
            + BOTTOM_PLATE_HEIGHT
    return createCaseOutline()\
        .faces()\
        .extrude(height)

### Bottom Plate

In [7]:
def buildBottomPlateRight():
    bottomPlateRight = createBottomPlate()
    bottomPlateRight = cutHole(bottomPlateRight, (117.16, 100.71)) # Reset Button

    # Rubber Pads
    bottomPlateRight = cutHole(bottomPlateRight, (89.3, 47.24), diameter=10.1, depth=BOTTOM_PLATE_INDENTATION)
    bottomPlateRight = cutHole(bottomPlateRight, (167.16, 44.34), diameter=10.1, depth=BOTTOM_PLATE_INDENTATION)
    bottomPlateRight = cutHole(bottomPlateRight, (88.28, 115.6), diameter=10.1, depth=BOTTOM_PLATE_INDENTATION)
    bottomPlateRight = cutHole(bottomPlateRight, (187.00, 96.00), diameter=10.1, depth=BOTTOM_PLATE_INDENTATION)
    
    # Mounting Holes
    for hole in HOLES:
        bottomPlateRight = bottomPlateRight.faces("<Z[2]")\
            .workplane(centerOption="ProjectedOrigin", origin=(hole[0][0], -hole[0][1], 0))\
            .cskHole(2, 3, 82, depth=None)

    return bottomPlateRight

def buildBottomPlateLeft():
    return buildBottomPlateRight().mirror(mirrorPlane="ZY", basePointVector=(0, 0, 0))

### Wired Case

In [8]:
def cutMcuAndAudioJackIndentation(case):
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT
    workplane = case.faces("<Z").workplane()
    cutout = workplane.moveTo(82.78,88.81)\
        .lineTo(82.78, 96.18)\
        .lineTo(101.51, 96.18)\
        .lineTo(101.51, 88.81)\
        .close()\
        .extrude(-(0.2 + PCB_SWITCH_PLATE_SPACING), combine=False)\
        .translate((0, 0, zOffset))
    audioJackIndentation = workplane.moveTo(82.78,82.42)\
        .lineTo(82.78, 88.81)\
        .lineTo(101.51, 88.81)\
        .lineTo(101.51, 82.42)\
        .close()\
        .extrude(-(5.5 + PCB_SWITCH_PLATE_SPACING), combine=False)\
        .translate((0, 0, zOffset))
    mcuIndentation = workplane.moveTo(82.78,42.26)\
        .lineTo(82.78, 82.42)\
        .lineTo(101.51, 82.42)\
        .lineTo(101.51, 42.26)\
        .close()\
        .extrude(-(OLED_SPACING + 1), combine=False)\
        .translate((0, 0, zOffset))
    
    return case.cut(cutout).cut(audioJackIndentation).cut(mcuIndentation)

def cutAcrylicGlassWindow(case):
    top = case.faces(">Z").vertices().first().val().Center().z - ACRYLIC_GLASS_HEIGHT
    cutout = cq.Workplane("XY")\
        .moveTo(86.38, -48.14)\
        .lineTo(86.38, -75.14)\
        .lineTo(98.38, -75.14)\
        .lineTo(98.38, -48.14)\
        .close()\
        .extrude(ACRYLIC_GLASS_HEIGHT, combine = False)\
        .translate((0, 0, top))
    return case.cut(cutout)

def cutAudioJackHole(case, rightSide):
    jackRadius = 2.5
    shieldPcbHeight = 0.6
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + jackRadius - shieldPcbHeight
    yOffset = -83.81 if rightSide else -85.55
    side = case.faces(">X[0]").workplane(centerOption="ProjectedOrigin", origin=(82.77, yOffset, zOffset))
    inner = side.circle(2.5).extrude(-5,combine=False)
    outer = side.circle(3.7).extrude(-2,combine=False)
    slideCutout = cq.Workplane("XY").moveTo(82.00, yOffset - jackRadius)\
        .lineTo(82.00, yOffset + jackRadius)\
        .lineTo(83.80, yOffset + jackRadius)\
        .lineTo(83.80, yOffset - jackRadius)\
        .close()\
        .extrude(zOffset + jackRadius, combine=False)
    
    return case.cut(inner).cut(outer).cut(slideCutout)

def buildWiredBaseCase():
    displayExtrusion = OLED_SPACING + ACRYLIC_GLASS_HEIGHT - SWITCH_BORDER_HEIGHT - PCB_SWITCH_PLATE_SPACING - SWITCH_PLATE_HEIGHT
    case = createBaseCase()
    case = extrudeDisplayCover(case, displayExtrusion).edges().fillet(1.9)
    case = cutPcb(case)
    case = cutKeycapsBorder(case)
    case = cutSwitchHoles(case)
    case = cutBottomPlate(case)
    case = cutTrackPointShieldIndentation(case)
    case = cutAcrylicGlassWindow(case)
    case = cutUsbCConnector(case, WIRED_MCU_USB_OFFSET)
    case = cutMcuAndAudioJackIndentation(case)
    case = cutMountingHoles(case)
    case = cutTrackpointHole(case)
    return case

def buildWiredCaseRight():
    case = buildWiredBaseCase()
    return cutAudioJackHole(case, rightSide=True)

def buildWiredCaseLeft():
    case = buildWiredBaseCase()
    case = cutAudioJackHole(case, rightSide=False)
    return case.mirror(mirrorPlane="ZY", basePointVector=(0, 0, 0))

### Wireless Case

In [9]:
def cutLcd(case):
    lcdWith = 14
    lcdHeight = 32
    lcdHeightExtra = 5 # for bending the FCC
    top = case.faces(">Z").vertices().first().val().Center().z
    lcd = cq.Workplane("XY", origin=(0, 0, 0))\
        .box(lcdWith, lcdHeight + lcdHeightExtra, LCD_COVER_HEIGHT, centered=False)\
        .faces(">Z")\
        .workplane(origin = (1.6, lcdHeight + lcdHeightExtra - 25.2 - 1.6, 0))\
        .box(10.7, 25.2, 1, centered=False)\
        .translate((85.38, -(75 + lcdHeightExtra), top - (1 + LCD_COVER_HEIGHT)))
    return case.cut(lcd)

def cutMWirelessMcuAndPowerSwitchIndentation(case):
    top = case.faces(">Z").vertices().first().val().Center().z
    top = case.faces(">Z").vertices().first().val().Center().z
    zOffset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT
    leverDistance = 2.2
    workplane = case.faces("<Z").workplane()
    cutout = workplane.moveTo(82.78,88.81)\
        .lineTo(82.78, 96.18)\
        .lineTo(101.51, 96.18)\
        .lineTo(101.51, 88.81)\
        .close()\
        .extrude(-2.6, combine=False)\
        .translate((0, 0, zOffset))
    audioJackIndentation = workplane.moveTo(82.78,82.42)\
        .lineTo(82.78, 88.81)\
        .lineTo(101.51, 88.81)\
        .lineTo(101.51, 82.42)\
        .close()\
        .extrude(-2.6, combine=False)\
        .translate((0, 0, zOffset))
    mcuIndentation = workplane.moveTo(82.78,42.26)\
        .lineTo(82.78, 82.42)\
        .lineTo(101.51, 82.42)\
        .lineTo(101.51, 42.26)\
        .close()\
        .extrude(-(top - (1 + LCD_COVER_HEIGHT)),combine=False)
    switch = cq.Workplane("XY")\
        .moveTo(82.63, -87.49)\
        .lineTo(82.63, -97.19)\
        .lineTo(88.14, -97.19)\
        .lineTo(88.14, -87.49)\
        .close()\
        .extrude(BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT + 4, combine=False)
    s1 = Sketch().rect(3.5,2.2).vertices().fillet(0.2)
    s2 = Sketch().rect(9, 6).vertices().fillet(0.2)
    switchLeverCurout = result = cq.Workplane("YZ")\
        .placeSketch(s1, s2.moved(Location(Vector(0, 0, 8))))\
        .loft()\
        .rotate((0,0,0), (0,1,0), 180).translate((82.92, -92.28, zOffset + leverDistance))

    return case.cut(cutout)\
            .cut(audioJackIndentation)\
            .cut(mcuIndentation)\
            .cut(switch)\
            .cut(switchLeverCurout)

def buildWirelessCaseRight():
    offset = BOTTOM_PLATE_HEIGHT + PCB_BOTTOM_SPACING + PCB_HEIGHT + PCB_SWITCH_PLATE_SPACING + SWITCH_PLATE_HEIGHT + 5 + 1 + 1
    topCutSolid = cq.Workplane("XY").box(400, 400, 200).translate((0, 0, offset+100))
    
    caseWireless = createBaseCase()
    caseWireless = extrudeDisplayCover(caseWireless, 12 - SWITCH_BORDER_HEIGHT).edges().fillet(2)
    caseWireless = caseWireless.cut(topCutSolid)
    caseWireless = cutPcb(caseWireless)
    caseWireless = cutKeycapsBorder(caseWireless)
    caseWireless = cutSwitchHoles(caseWireless)
    caseWireless = cutBottomPlate(caseWireless)
    caseWireless = cutTrackPointShieldIndentation(caseWireless)
    caseWireless = cutUsbCConnector(caseWireless, WIRELESS_MCU_USB_OFFSET, outerDepth=-10)
    caseWireless = cutMountingHoles(caseWireless)
    caseWireless = cutLcd(caseWireless)
    caseWireless = cutMWirelessMcuAndPowerSwitchIndentation(caseWireless)
    return caseWireless

def buildWirelessCaseLeft():
    case = buildWirelessCaseRight()
    return case.mirror(mirrorPlane="ZY", basePointVector=(0, 0, 0))

## Build & Export

### Bottom Plate

In [10]:
bottomPlateRight = buildBottomPlateRight()
bottomPlateLeft = buildBottomPlateLeft()

In [11]:
exporters.export(bottomPlateRight, 'bottomPlate_right.stl')
exporters.export(bottomPlateLeft, 'bottomPlate_left.stl')

### Wired Case

In [12]:
wiredCaseRight = buildWiredCaseRight()
wiredCaseLeft = buildWiredCaseLeft()

In [13]:
exporters.export(wiredCaseRight, 'case_wired_right.stl')
exporters.export(wiredCaseLeft, 'case_wired_left.stl')

In [14]:
all = (cq.Assembly()\
       .add(wiredCaseRight, name="caseRight", color=cq.Color("paleturquoise3"))\
       .add(wiredCaseLeft, name="caseLeft", color=cq.Color("paleturquoise3"))\
       .add(bottomPlateRight, name="plate", color=cq.Color("paleturquoise4"))
      )
show(all)

100% ⋮————————————————————————————————————————————————————————————⋮ (3/3)  0.89s


### Wireless Case

In [ ]:
wirelessCaseRight = buildWirelessCaseRight()
wirelessCaseLeft = buildWirelessCaseLeft()

In [ ]:
exporters.export(wirelessCaseRight, 'case_wireless_right.stl')
exporters.export(wirelessCaseLeft, 'case_wireless_left.stl')

In [ ]:
all = (cq.Assembly()\
       .add(wirelessCaseRight, name="caseRight", color=cq.Color("paleturquoise3"))\
       .add(wirelessCaseLeft, name="caseLeft", color=cq.Color("paleturquoise3"))\
       .add(bottomPlateRight, name="plate", color=cq.Color("paleturquoise4"))
      )
show(all)